In [105]:
'''Current issues for all functions:
    Fair Price Error (Make it so that is bid or ask is 0, fair price is 0.01)
    Get float to round upto 2 decimal places
    Make the entire thing wayyy more efficient
'''
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
import numpy as np
import yfinance as yf
import pytz
import datetime
from os import system, name

date = '2020-08-20'
s = yf.Ticker("UCO")
# print(s.history(period = "max")['Close'].iloc[-1])
currentPrice = s.history(period = "max")['Close'].iloc[-1]
oldPrice = s.history(period = "max")['Low'].iloc[-20]
priceRange = currentPrice - oldPrice


opt = s.option_chain(date)
calls, puts = opt.calls, opt.puts
calls = call[call['strike'] in range (int(currentPrice - priceRange), int(currentPrice - priceRange))]
print(calls)

print(int(calls['volume'].mean()))
calls['Fair Price'] = [i if i >= 0.01 else 0.01 for i in (calls['ask']+calls['bid'])/2]
puts['Fair Price']  = [i if i >= 0.01 else 0.01 for i in (puts['ask'] + puts['bid'])/2]


False
10


In [98]:
print(calls[calls['strike'] >= (currentPrice - priceRange)])

        contractSymbol       lastTradeDate  strike  lastPrice   bid   ask  \
13  UCO200821C00028000 2020-07-10 18:35:27  28.000      4.800 4.700 4.850   
14  UCO200821C00029000 2020-07-16 15:36:44  29.000      4.470 4.050 4.200   
15  UCO200821C00030000 2020-07-15 16:41:58  30.000      3.700 3.400 3.600   
16  UCO200821C00031000 2020-07-16 16:49:30  31.000      2.950 2.880 2.990   
17  UCO200821C00032000 2020-07-16 18:12:41  32.000      2.460 2.370 2.510   
18  UCO200821C00033000 2020-07-16 17:51:53  33.000      2.020 1.970 2.080   
19  UCO200821C00034000 2020-07-16 15:25:07  34.000      1.950 1.600 1.720   
20  UCO200821C00035000 2020-07-16 15:31:21  35.000      1.550 1.300 1.380   
21  UCO200821C00036000 2020-07-15 16:51:13  36.000      1.250 1.040 1.130   
22  UCO200821C00037000 2020-07-16 15:16:32  37.000      1.020 0.840 0.920   
23  UCO200821C00038000 2020-07-10 18:49:59  38.000      0.880 0.680 0.770   
24  UCO200821C00039000 2020-07-10 13:30:00  39.000      0.690 0.560 0.650   

In [78]:

def getCalls():
    c = calls[calls['volume']>=int(calls['volume'].mean())]
#     if t == 'Credit':
#         c = calls[calls['inTheMoney'] == False]
    l = c[['strike', 'Fair Price']]
    j = c[['strike', 'lastPrice']]
    if c['ask'].mean() != 0: #need to use time library for 
        m = l.set_index('strike').to_dict()
        n = m['Fair Price'] 
        print("Using Current Market Price")
    else:
        m = j.set_index('strike').to_dict()
        n = m['lastPrice']
        print("Using Last Price")
    return n

def getPuts():
    p = puts[puts['volume']>=int(puts['volume'].mean())]
    l = p[['strike', 'Fair Price']]
    j = p[['strike', 'lastPrice']]
    if p['ask'].mean() != 0:
        m = l.set_index('strike').to_dict()
        n = m['Fair Price'] 
        print("Using Current Market Price")
    else:
        m = j.set_index('strike').to_dict()
        n = m['lastPrice']
        print("Using Last Price")
    return n

def appendBest(best_ratio, l, s, lp, sp, r, mrx, mrw):
    best_ratio['Long(Buy)']=l
    best_ratio['Short(Sell)']=s
    best_ratio['Risk/Reward Ratio']=r
    best_ratio['Short Premium'] =sp
    best_ratio['Long Premium'] = lp
    best_ratio['maxRisk'] = mrx
    best_ratio['maxReward'] = mrw

    
def rnR(type, sL, sS, pL, pS):
    if type == 'callC' or type == 'putC':
        MaxReward = pS - pL
        MaxRisk = (sL - sS - MaxReward)
        if type == 'putC':
            MaxRisk *= -1
    if type == 'callD' or type == 'putD':
#         print(sL,  sS)
        MaxRisk = pL - pS
#         print(MaxRisk)
        MaxReward = (sL - sS) - MaxRisk
        if type == 'callD':
            MaxReward *= -1
    return MaxRisk, MaxReward



In [79]:
def Put_Debit_Spread():
    
    n = getPuts()

#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio

    # best_ratio: Dictionary to maintain the best available trade at a given time
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    
    # First loop to check what we're buying
    for long in n:
        
        # Second loop for what we're selling
        for short in n:
            
            # We don't want buying strike to be smaller than selling strike (Put debit, buy higher strike and sell lower strike)
            if long <= short:
                continue
            else:
                
                #Declaring variables
                strikeShort = short
                strikeLong = long
                premiumShort = n[short]
                premiumLong = n[long]
                
                # figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                z = rnR('putD', strikeLong, strikeShort, premiumLong, premiumShort)
#                 print("\n", z)
                MaxRisk = z[0]
                MaxReward = z[1]
                
                
                if MaxReward>0:
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        appendBest(best_ratio, long, short, premiumLong, premiumShort, Ratio, MaxRisk, MaxReward)


                        
    return print("Best Put Debit Spread: ", best_ratio)


In [80]:
def Call_Credit_Spread():
    
    n = getCalls()
    
#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio

# best_ratio: Dictionary to maintain the best available trade at a given time
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}

    # First loop to check what to short (sell)   
    for short in n:
        
        # Second loop to check what we're buying (long)
        for long in n:
            
            # In a credit call spread the long Strike cannot be lower or equal to the short strike price
            if long <= short:
                continue
            else:
                
                # Declaring variables to work with
                strikeShort = short
                strikeLong = long
                
                premiumShort = n[short]
                premiumLong = n[long]
                
                # Figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                z = rnR('callC', strikeLong, strikeShort, premiumLong, premiumShort)
                MaxRisk = z[0]
                MaxReward = z[1]
                
                if MaxReward>0:
                    
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        appendBest(best_ratio, long, short, premiumLong, premiumShort, Ratio, MaxRisk, MaxReward)
                        
                        
    # returning the final best trade available as a call credit spread              
    return print("Best Call Credit Spread: ", best_ratio)
        


def Put_Credit_Spread():
    
    n = getPuts()
    
#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio

    # best_ratio: Dictionary to maintain the best available trade at a given time
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    
    # First loop to check what to long (buy) 
    for long in n:
        
        # Second loop to check what we're selling(short)
        for short in n:
            
            # In a put credit spread the strike price of what we're buying cannot be higher than strike price of what we're selling
            if long >= short:
                continue
                
            else:
                
                #Decalring various variables
                strikeShort = short
                strikeLong = long
                premiumShort = n[short]
                premiumLong = n[long]
                
                #figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                z = rnR('putC', strikeLong, strikeShort, premiumLong, premiumShort)
                MaxRisk = z[0]
                MaxReward = z[1]
                
                if MaxReward>0:
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        appendBest(best_ratio, long, short, premiumLong, premiumShort, Ratio, MaxRisk, MaxReward)


                        
                        
    return print("Best Put Credit Spread: ", best_ratio)




def Call_Debit_Spread():
    
    n = getCalls()

#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    
    
    # First loop to check what we're buying
    for long in n:
        
        # Second loop for what we're selling
        for short in n:
            
            # We don't want buying strike to be higher than selling strike (Call debit, sell higher strike and buy lower strike)
            if long >= short:
                continue
            else:
                
                #Declaring variables
                strikeShort = short
                strikeLong = long
                premiumShort = n[short]
                premiumLong = n[long]
                

                z = rnR('callD', strikeLong, strikeShort, premiumLong, premiumShort)
                MaxRisk = z[0]
                MaxReward = z[1]
                
                                   
                if MaxReward > 0:
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        appendBest(best_ratio, long, short, premiumLong, premiumShort, Ratio, MaxRisk, MaxReward)


                        
                        
    return print("Best Call Debit Spread: ", best_ratio)


In [81]:
def finalSpreads():
    print("\t\t~~~~~~~~~~~~~~~~~~~~~~~ Bullish for Date",date , "~~~~~~~~~~~~~~~~~~~~~~~~~ \n")
    Call_Debit_Spread()
    print("\n")
    Put_Credit_Spread()
    print("\n")
    print("\t\t~~~~~~~~~~~~~~~~~~~~~~~ Bearish for Date",date ,"~~~~~~~~~~~~~~~~~~~~~~~~~~ \n")
    Put_Debit_Spread()
    print("\n")
    Call_Credit_Spread()


finalSpreads()

		~~~~~~~~~~~~~~~~~~~~~~~ Bullish for Date 2020-08-20 ~~~~~~~~~~~~~~~~~~~~~~~~~ 

Using Current Market Price
Best Call Debit Spread:  {'Short(Sell)': 42.0, 'Long(Buy)': 40.0, 'Risk/Reward Ratio': 0.058823529411764705, 'Short Premium': 0.39, 'Long Premium': 0.515, 'maxRisk': 0.125, 'maxReward': 2.125}


Using Current Market Price
Best Put Credit Spread:  {'Short(Sell)': 34.0, 'Long(Buy)': 33.0, 'Risk/Reward Ratio': 2.249999999999999, 'Short Premium': 4.775, 'Long Premium': 3.9749999999999996, 'maxRisk': 1.8000000000000007, 'maxReward': 0.8000000000000007}


		~~~~~~~~~~~~~~~~~~~~~~~ Bearish for Date 2020-08-20 ~~~~~~~~~~~~~~~~~~~~~~~~~~ 

Using Current Market Price
Best Put Debit Spread:  {'Short(Sell)': 14.0, 'Long(Buy)': 20.0, 'Risk/Reward Ratio': 0.048951048951048945, 'Short Premium': 0.2, 'Long Premium': 0.48, 'maxRisk': 0.27999999999999997, 'maxReward': 5.72}


Using Current Market Price
Best Call Credit Spread:  {'Short(Sell)': 20.0, 'Long(Buy)': 25.0, 'Risk/Reward Ratio': 0.15606